In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import string
import random
import re
import csv
import pickle
from unidecode import unidecode

2024-01-07 18:50:05.612152: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-07 18:50:05.612193: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-07 18:50:05.613587: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-07 18:50:05.620015: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-07 18:50:07.093202: W tensorflow/compiler/tf2

In [2]:
# JOKES DATASET PREPARATION
data = []
with open("jokes.csv") as f:
    x = csv.reader(f)
    f = 0;
    for i in x:
        if f:
            data.append([
                i[1], 
                '[start] ' + i[2] + ' [end]'
            ])
        f = 1;
data = np.array(data )
print(data[:2])

[['Did you hear about the Native American man that drank 200 cups of tea?'
  '[start] He nearly drown in his own tea pee. [end]']
 ["What's the best anti diarrheal prescription?"
  '[start] Mycheexarphlexin [end]']]


In [3]:
print(data[120])
print(data.shape)

["What's 11 & 2?" '[start] The Cowboys [end]']
(38269, 2)


In [4]:
MAX_TOKENS = 20000
MAX_SEQUENCE_LENGTH = 100
EMBEDDING_DIM = 128
DENSE_DIM = 512
BATCH_SIZE = 16
NUM_HEADS = 8
EPOCHS = 10

In [5]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "") 
strip_chars = strip_chars.replace("]", "")

def custom_standardization(s):
    lowercase = tf.strings.lower(s)
    return tf.strings.regex_replace(lowercase, f'[{re.escape(strip_chars)}]', '')

question_vectorizer = keras.layers.TextVectorization(
    max_tokens=MAX_TOKENS,
    output_mode="int",
    output_sequence_length=MAX_SEQUENCE_LENGTH,
    standardize=custom_standardization,
)

answer_vectorizer = keras.layers.TextVectorization(
    max_tokens=MAX_TOKENS,
    output_mode="int",
    output_sequence_length=MAX_SEQUENCE_LENGTH + 1,
    standardize=custom_standardization,
)

question_data = data[:, 0]
answer_data = data[:, 1]
question_vectorizer.adapt(question_data)
answer_vectorizer.adapt(answer_data)

2024-01-07 18:50:09.281271: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-07 18:50:09.323136: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-07 18:50:09.323598: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [6]:
print(question_data[0])
print(answer_data[0])
print(len(question_data))
print(len(answer_data))

Did you hear about the Native American man that drank 200 cups of tea?
[start] He nearly drown in his own tea pee. [end]
38269
38269


In [7]:
def prepare_dataset(questions, answers):
    tokenized_questions = question_vectorizer(questions)
    tokenized_answers = answer_vectorizer(answers)
    print(tokenized_questions.shape, tokenized_answers.shape)
    x = {"question": tokenized_questions, "answer": tokenized_answers[:, :-1]}
    y = tokenized_answers[: , 1:]
    return (x, y)

full_data_ds = tf.data.Dataset.from_tensor_slices((question_data, answer_data))
full_data_ds = full_data_ds.batch(BATCH_SIZE)
full_data_ds = full_data_ds.shuffle(buffer_size=4096)
full_data_ds = full_data_ds.map(prepare_dataset, num_parallel_calls=8)
full_data_ds = full_data_ds.prefetch(16).cache()   

(None, 100) (None, 101)


In [8]:
for i in full_data_ds.take(1):
    print(i[0]['question'].shape)
    print(i[0]['answer'].shape)
    print(i[1].shape)
    print(i[0]['question'][0])
    print(i[0]['answer'][0])
    print(i[1][0])

(16, 100)
(16, 100)
(16, 100)
tf.Tensor(
[   4    3   42  676   24  552   10 5400    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0], shape=(100,), dtype=int64)
tf.Tensor(
[   2 1123    3    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0 

2024-01-07 18:50:11.692956: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [9]:
# # remove from memory
del data
del question_data
del answer_data

In [10]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads

        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation='relu'),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            'embed_dim': self.embed_dim,
            'dense_dim': self.dense_dim,
            'num_heads': self.num_heads,
        })
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads

        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim       
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation='relu'),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.support_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            'embed_dim': self.embed_dim,
            'dense_dim': self.dense_dim,
            'num_heads': self.num_heads,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i>=j, dtype='int32')
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0
        )
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype='int32'
            )
            padding_mask = tf.minimum(padding_mask,  causal_mask)

        attention_output_1 = self.attention_1(
            query = inputs,
            value = inputs, 
            key = inputs,
            attention_mask = causal_mask
        )
        attention_output_1 = self.layernorm_1(
            attention_output_1 + inputs
        )
        attention_output_2 = self.attention_2(
            query = attention_output_1,
            value = encoder_outputs,
            key = encoder_outputs,
            attention_mask = padding_mask
        )
        attention_output_2 = self.layernorm_2(
            attention_output_2 + attention_output_1
        )
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(proj_output + attention_output_2)

    
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim
        )
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            'sequence_length': self.sequence_length,
            'input_dim': self.input_dim,
            'output_dim': self.output_dim,
        })
        return config

In [11]:
encoder_inputs = keras.Input(shape=(None,), dtype='int64', name='question')
x = PositionalEmbedding(MAX_SEQUENCE_LENGTH, MAX_TOKENS, EMBEDDING_DIM)(encoder_inputs)
encoder_outputs = TransformerEncoder(EMBEDDING_DIM, DENSE_DIM, NUM_HEADS)(x)

decoder_inputs = keras.Input(shape=(None,), dtype='int64', name='answer')
x = PositionalEmbedding(MAX_SEQUENCE_LENGTH, MAX_TOKENS, EMBEDDING_DIM)(decoder_inputs)
x = TransformerDecoder(EMBEDDING_DIM, DENSE_DIM, NUM_HEADS)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(MAX_TOKENS, activation='softmax')(x)

transformer = keras.Model(
    inputs = [encoder_inputs, decoder_inputs],
    outputs = decoder_outputs
)

transformer.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 question (InputLayer)       [(None, None)]               0         []                            
                                                                                                  
 answer (InputLayer)         [(None, None)]               0         []                            
                                                                                                  
 positional_embedding (Posi  (None, None, 128)            2572800   ['question[0][0]']            
 tionalEmbedding)                                                                                 
                                                                                                  
 positional_embedding_1 (Po  (None, None, 128)            2572800   ['answer[0][0]']          

In [12]:
class SaveModelCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        self.model.save('training_checkpoint/puns_bot.keras')
        # with open("training_checkpoint/puns_bot.pickle", 'wb') as f:
        #     pickle.dump(transformer, f)

callbacks = [SaveModelCallback()]

transformer.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [17]:
transformer.fit(
    full_data_ds, 
    epochs=50,
    callbacks=callbacks,
)

Epoch 1/50
2392/2392 [==============================] - 182s 76ms/step - loss: 0.3889 - accuracy: 0.9443
Epoch 2/50
2392/2392 [==============================] - 182s 76ms/step - loss: 0.3637 - accuracy: 0.9462
Epoch 3/50
2392/2392 [==============================] - 182s 76ms/step - loss: 0.3420 - accuracy: 0.9480
Epoch 4/50
2392/2392 [==============================] - 182s 76ms/step - loss: 0.3238 - accuracy: 0.9497
Epoch 5/50
2392/2392 [==============================] - 182s 76ms/step - loss: 0.3116 - accuracy: 0.9508
Epoch 6/50
2392/2392 [==============================] - 182s 76ms/step - loss: 0.2963 - accuracy: 0.9524
Epoch 7/50
2392/2392 [==============================] - 182s 76ms/step - loss: 0.2840 - accuracy: 0.9536
Epoch 8/50
2392/2392 [==============================] - 182s 76ms/step - loss: 0.2739 - accuracy: 0.9546
Epoch 9/50
2392/2392 [==============================] - 182s 76ms/step - loss: 0.2648 - accuracy: 0.9555
Epoch 10/50
2392/2392 [==============================] 

In [59]:
inp = "Why did the storm trooper decide to buy an iPhone?"
inp = question_vectorizer([inp])

target_dict = dict([(i, val) for i, val in enumerate(answer_vectorizer.get_vocabulary())])

output = "[start]"
for i in range(MAX_SEQUENCE_LENGTH):
    x = answer_vectorizer([output])[:, :-1]
    x = transformer([inp, x])
    x = x[:, i, :]
    x = tf.argmax(x, axis=-1)
    x = target_dict[x.numpy()[0]]
    output += " " + x
    if x == "[end]":
        break

print(output)

[start] because he always made the droid he was looking for [end]


In [63]:
inp = "What did Tom hanks do in the woods?"
inp = question_vectorizer([inp])

target_dict = dict([(i, val) for i, val in enumerate(answer_vectorizer.get_vocabulary())])

output = "[start]"
for i in range(MAX_SEQUENCE_LENGTH):
    x = answer_vectorizer([output])[:, :-1]
    x = transformer([inp, x])
    x = x[:, i, :]
    x = tf.argmax(x, axis=-1)
    x = target_dict[x.numpy()[0]]
    output += " " + x
    if x == "[end]":
        break

print(output)

[start] he took a vow of pavarotti [end]


In [67]:
inp = "What do A Tribe Called Quest and margarine have in common?"
inp = question_vectorizer([inp])

target_dict = dict([(i, val) for i, val in enumerate(answer_vectorizer.get_vocabulary())])

output = "[start]"
for i in range(MAX_SEQUENCE_LENGTH):
    x = answer_vectorizer([output])[:, :-1]
    x = transformer([inp, x])
    x = x[:, i, :]
    x = tf.argmax(x, axis=-1)
    x = target_dict[x.numpy()[0]]
    output += " " + x
    if x == "[end]":
        break

print(output)

[start] its like butter baby [end]


In [68]:
inp = "Why did the three year old African boy buy a red convertible?"
inp = question_vectorizer([inp])

target_dict = dict([(i, val) for i, val in enumerate(answer_vectorizer.get_vocabulary())])

output = "[start]"
for i in range(MAX_SEQUENCE_LENGTH):
    x = answer_vectorizer([output])[:, :-1]
    x = transformer([inp, x])
    x = x[:, i, :]
    x = tf.argmax(x, axis=-1)
    x = target_dict[x.numpy()[0]]
    output += " " + x
    if x == "[end]":
        break

print(output)

[start] he was having a midlife crisis [end]
